In [13]:
import requests
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
import bs4 as bs
import numpy as np
import re

In [2]:
#restaurant for a given place/address
#headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
         'accept-encoding': 'gzip, deflate, br',
         'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
         'accept-language': 'en-US,en;q=0.9',
         'if-none-match': 'W/"18a70-2svTzwERwbNJVGpuh7Z+/vbtj8U"'}

In [3]:
arr1=[[]]
lat = 12.8933808
long = 77.6609237
r1= requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=0&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
r1_js = r1.json()
tot_rest_cnt = r1_js['data']['totalSize']

for off in range(0, tot_rest_cnt, 14):
    r1=requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=' + str(lat) + '&lng=' + str(long) + '&offset=' + str(off) +'&sortBy=RELEVANCE&pageType=SEE_ALL&page_type=DESKTOP_SEE_ALL_LISTING', headers= headers)
    r1_js = r1.json()    
    try:
        rn = len(r1_js['data']['cards'])
    except:
        break
    for i in range(rn):
        cardType = r1_js['data']['cards'][i]['cardType']
        if cardType == 'restaurant':    
            name = r1_js['data']['cards'][i]['data']['data']['name']
            rest_id = r1_js['data']['cards'][i]['data']['data']['id']
            city = r1_js['data']['cards'][i]['data']['data']['city']
            area = r1_js['data']['cards'][i]['data']['data']['area']
            rating = r1_js['data']['cards'][i]['data']['data']['avgRating']
            cuisine = r1_js['data']['cards'][i]['data']['data']['cuisines']
            cost_for_two = r1_js['data']['cards'][i]['data']['data']['costForTwo']
            del_time = r1_js['data']['cards'][i]['data']['data']['deliveryTime']
            min_del_time = r1_js['data']['cards'][i]['data']['data']['minDeliveryTime']
            max_del_time = r1_js['data']['cards'][i]['data']['data']['maxDeliveryTime']
            loc = r1_js['data']['cards'][i]['data']['data']['locality']
            #discount attributes
            try:
                disc_meta = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['meta']
                disc_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['discountType']
                disc_op_type = r1_js['data']['cards'][i]['data']['data']['aggregatedDiscountInfo']['shortDescriptionList'][0]['operationType']
            except:
                disc_meta = '' 
                disc_type = ''
                disc_op_type = '' 

            #fee details
            fee = r1_js['data']['cards'][i]['data']['data']['feeDetails']['totalFees']
            try:
                menu_link = r1_js['data']['cards'][i]['data']['data']['cta']['link']
            except:
                menu_link = ''
            slug = r1_js['data']['cards'][i]['data']['data']['slugs']['restaurant']
            #making an array of the above
            arr = np.array([name, rest_id, city, area, rating, cuisine, cost_for_two, del_time, max_del_time, min_del_time, loc, disc_meta,
                   disc_type, disc_op_type, fee, menu_link, slug])
            try:
                arr1 = np.append(arr1, [arr], axis=0)
            except:
                arr1 = [arr]

C:\Users\hp\anaconda3\lib\site-packages\ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
columns=['name', 'rest_id', 'city', 'area', 'rating', 'cuisine', 'cost_for_two', 'del_time', 'max_del_time', 'min_del_time', 
         'loc', 'disc_meta', 'disc_type', 'disc_op_type', 'fee', 'menu_link', 'slug']
pd.DataFrame(arr1, columns=columns).to_csv('C:\\Users\\hp\\Desktop\\My Projects\\Restaurants\\202001112253_purva.csv')

In [4]:
columns=['name', 'rest_id', 'city', 'area', 'rating', 'cuisine', 'cost_for_two', 'del_time', 'max_del_time', 'min_del_time', 
         'loc', 'disc_meta', 'disc_type', 'disc_op_type', 'fee', 'menu_link', 'slug']
df_rest_list = pd.DataFrame(arr1, columns=columns)

## Restaurant Menu

In [ ]:
r1= requests.get('https://www.swiggy.com/dapi/restaurants/list/v5?lat=12.8933808&lng=77.6609237&collection=46147&offset=800&pageType=COLLECTION&type=rcv2&page_type=DESKTOP_COLLECTION_LISTING', headers= headers)

In [ ]:
r1.json()

In [5]:
df_rest_list

,name,rest_id,city,area,rating,cuisine,cost_for_two,del_time,max_del_time,min_del_time,loc,disc_meta,disc_type,disc_op_type,fee,menu_link,slug
0,XO Belgian Waffle,79027,1,HSR,4.3,[Desserts],25000,28,28,28,HSR Layout,30% off | Use SWIGGYIT,Percentage,RESTAURANT,0,,xo-belgian-waffle-hbr-layout-kammanahallikalya...
1,Singh Da Pind,294144,1,HSR,4.1,"[Punjabi, North Indian, Chinese, Mughlai]",40000,28,28,28,HSR Layout,30% off | Use SWIGGYIT,Percentage,RESTAURANT,4000,,singh-da-pind-kudlu-gate-kudlu-gate
2,Only Maggi,196037,1,HSR,--,"[Fast Food, North Indian]",20000,25,25,25,SOMASANDARA PALYA,,,,4000,,only-maggi-rajarajeshwari-nagar-rajarajeshwari...
3,WarmOven Cake & Desserts,27996,1,HSR,4.3,"[Bakery, Desserts, Ice Cream]",20000,31,31,31,HSR,60% off | Use DEAL60,Percentage,RESTAURANT,0,,warm-oven-hsr-hsr
4,Vijayawada Style,157341,1,Electronic City,2.9,[Indian],36000,34,34,34,"electronics city, Phase 2",30% off on orders above ₹189,Percentage,RESTAURANT,4000,,vijayawada-style-banaswadi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Yum In My Tum,107301,1,BTM,3.7,"[Chinese, Continental, Beverages, Pizzas, Ital...",20000,57,57,57,2nd Stage,50% off | Use SWIGGYIT,Percentage,RESTAURANT,0,,yum-in-my-tum-bengaluru-karnataka-btm
152,Juice Shop,246129,1,Electronic City,4.2,"[Fast Food, Juices]",15000,56,56,56,Electronic City,50% off | Use SWIGGYIT,Percentage,RESTAURANT,0,,juice-shop-electronic-city-electronic-city
153,Polar Bear,239395,1,BTM,4.1,"[Desserts, Ice Cream]",30000,53,53,53,BTM,,,,4000,,polar-bear-btm-btm
154,Delhi Wala Paratha,230443,1,BTM,--,[North Indian],20000,52,52,52,9th D Main Cross,,,,4000,,delhi-wala-paratha-btm-btm


In [10]:
rest_slug = df_rest_list['slug'][1] + '-' + df_rest_list['rest_id'][1]
rest_url = 'https://www.swiggy.com/restaurants/' + rest_slug
mr = requests.get(rest_url, headers = headers)

In [30]:
mr_text = re.split('window.webpackManifest', re.split('INITIAL_STATE__ = ', mr.text)[1])[0].strip().replace(';', '')
mr_json = json.loads(mr_text)

In [46]:
for item in list(dict.keys(mr_json['menu']['items'])):
    item_id = mr_json['menu']['items'][item]['id']
    name = mr_json['menu']['items'][item]['name']
    category = mr_json['menu']['items'][item]['category']
    stock = mr_json['menu']['items'][item]['inStock']
    price = mr_json['menu']['items'][item]['price']/100
    vegClassifier = mr_json['menu']['items'][item]['attributes']['vegClassifier']
    menu_temp = np.array([category, name, stock, price, vegClassifier, item_id])
    try:
        menu = np.append(menu, [menu_temp], axis=0)
    except:
        menu = [menu_temp]

In [48]:
df_rest_list

,name,rest_id,city,area,rating,cuisine,cost_for_two,del_time,max_del_time,min_del_time,loc,disc_meta,disc_type,disc_op_type,fee,menu_link,slug
0,XO Belgian Waffle,79027,1,HSR,4.3,[Desserts],25000,28,28,28,HSR Layout,30% off | Use SWIGGYIT,Percentage,RESTAURANT,0,,xo-belgian-waffle-hbr-layout-kammanahallikalya...
1,Singh Da Pind,294144,1,HSR,4.1,"[Punjabi, North Indian, Chinese, Mughlai]",40000,28,28,28,HSR Layout,30% off | Use SWIGGYIT,Percentage,RESTAURANT,4000,,singh-da-pind-kudlu-gate-kudlu-gate
2,Only Maggi,196037,1,HSR,--,"[Fast Food, North Indian]",20000,25,25,25,SOMASANDARA PALYA,,,,4000,,only-maggi-rajarajeshwari-nagar-rajarajeshwari...
3,WarmOven Cake & Desserts,27996,1,HSR,4.3,"[Bakery, Desserts, Ice Cream]",20000,31,31,31,HSR,60% off | Use DEAL60,Percentage,RESTAURANT,0,,warm-oven-hsr-hsr
4,Vijayawada Style,157341,1,Electronic City,2.9,[Indian],36000,34,34,34,"electronics city, Phase 2",30% off on orders above ₹189,Percentage,RESTAURANT,4000,,vijayawada-style-banaswadi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Yum In My Tum,107301,1,BTM,3.7,"[Chinese, Continental, Beverages, Pizzas, Ital...",20000,57,57,57,2nd Stage,50% off | Use SWIGGYIT,Percentage,RESTAURANT,0,,yum-in-my-tum-bengaluru-karnataka-btm
152,Juice Shop,246129,1,Electronic City,4.2,"[Fast Food, Juices]",15000,56,56,56,Electronic City,50% off | Use SWIGGYIT,Percentage,RESTAURANT,0,,juice-shop-electronic-city-electronic-city
153,Polar Bear,239395,1,BTM,4.1,"[Desserts, Ice Cream]",30000,53,53,53,BTM,,,,4000,,polar-bear-btm-btm
154,Delhi Wala Paratha,230443,1,BTM,--,[North Indian],20000,52,52,52,9th D Main Cross,,,,4000,,delhi-wala-paratha-btm-btm


In [52]:
#putting together the fragments of code for the full menu
for i in range(len(df_rest_list)):
    rest_slug = df_rest_list['slug'][i] + '-' + df_rest_list['rest_id'][i]
    rest_url = 'https://www.swiggy.com/restaurants/' + rest_slug
    rest_name = df_rest_list['name'][i]
    
    mr = requests.get(rest_url, headers = headers)
    mr_text = re.split('window.webpackManifest', re.split('INITIAL_STATE__ = ', mr.text)[1])[0].strip().replace(';', '')
    mr_json = json.loads(mr_text)

    for item in list(dict.keys(mr_json['menu']['items'])):
        item_id = mr_json['menu']['items'][item]['id']
        name = mr_json['menu']['items'][item]['name']
        category = mr_json['menu']['items'][item]['category']
        stock = mr_json['menu']['items'][item]['inStock']
        price = mr_json['menu']['items'][item]['price']/100
        try:
            vegClassifier = mr_json['menu']['items'][item]['attributes']['vegClassifier']
        except:
            vegClassifier =''
        menu_temp = np.array([rest_name, category, name, stock, price, vegClassifier, item_id])
        try:
            menu = np.append(menu, [menu_temp], axis=0)
        except:
            menu = [menu_temp]


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-52-6e036ca437f5>", line 7, in <module>
    mr = requests.get(rest_url, headers = headers)
  File "C:\Users\hp\anaconda3\lib\site-packages\requests\api.py", line 75, in get
    return request('get', url, params=params, **kwargs)
  File "C:\Users\hp\anaconda3\lib\site-packages\requests\api.py", line 60, in request
    return session.request(method=method, url=url, **kwargs)
  File "C:\Users\hp\anaconda3\lib\site-packages\requests\sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "C:\Users\hp\anaconda3\lib\site-packages\requests\sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "C:\Users\hp\anaconda3\lib\site-packages\requests\adapters.py", line 449, in send
    timeout=timeout
  File "C:\Users\hp\anaconda3\lib\s

TypeError: object of type 'NoneType' has no len()

In [54]:
menu

array([['Grub@99', 'Meals', 'Chapathi(3)+rice+subji+dal+salad+sweet',
        ..., '151.0', 'VEG', '46718276'],
       ['Grub@99', 'Meals', 'Aloo Payaz Paratha (2)+Subji+Curd+Pickle',
        ..., '160.0', 'VEG', '46718279'],
       ['Grub@99', 'Meals', 'Paneer Paratha (2)+Subji+Curd+Pickle', ...,
        '170.0', 'VEG', '46718283'],
       ...,
       ['Immunity Booster Meals by Lunchbox', 'Desserts',
        'Chocolate Crusted Cheesecake', ..., '150.0', 'NONVEG',
        '64711521'],
       ['Immunity Booster Meals by Lunchbox', 'Desserts',
        'Red Velvet Lava Cake (Pack of 2)', ..., '220.0', 'VEG',
        '64711522'],
       ['Immunity Booster Meals by Lunchbox', 'Desserts',
        'Banana lava cake (Pack of 2)', ..., '199.0', 'VEG', '64711524']],
      dtype='<U80')

In [51]:
mr_json['menu']['items'][item]

{'id': 41914669,
 'name': 'Plain Maggi',
 'category': "Veg Lover's",
 'description': '',
 'recommended': 0,
 'inStock': 1,
 'isVeg': 1,
 'enabled': 1,
 'displayOrder': 0,
 'price': 7000,
 'variants_new': {'exclude_list': [], 'variant_groups': []},
 'cropChoices': 2,
 'itemScore': 0,
 'itemDiscount': 0,
 'isPopular': 0,
 'restId': '196037',
 'showMC': 0,
 'itemNudgeType': ''}